In [1]:
import pandas as pd
import numpy as np

import time
from datetime import datetime

from random import randint

import re
import string

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.utils import shuffle

from tqdm import tqdm

import json

import paralleldots
paralleldots.set_api_key("Vo6Oh77LXmw5HXMu691RKQIZG03nY1zB9rsVTkhEHqE") # lrl818@qq.com
print('Current key:', 'Vo6Oh77LXmw5HXMu691RKQIZG03nY1zB9rsVTkhEHqE', 1)



keys = ['OwoCuujTTYS5o3Vrx0M4A7RevaHLs7uUHlUD3Afa0XI' # laualan@connect.hku.hk
        'DwGe3alGduDGJnE1FT3O112wMDNJhZDnGysR0KFfZgY', # rlalan@outlook.com
        # 'JIkATWugt8LIP3PbQrdm6cfm1hZ4DbouVoyw8oAQqhI', # 394414515@qq.com
        'Vf9mBAl7wp0s8Fl43E9aDtVPOZhftIAv0pYJvIlYGD4', # 806124854@qq.com
        # '8gAwuDOEE92zakGfDXc6PoCtqhVop0htEnEd4IHFe0U', # yleun.lau@gmail.com
        'xTNXoCxtbg36jCgT6ArKgyOPJtnJdf5QQpEEKYoSwu8', # alanconstantinelau@gmail.com
        'INsEQXjCWuaZJQyzEbHg5Kw6C8liJiHf2DXOHqmHG70', # 122075300@qq.com
        'jpctHohVPtW9QlezbIw4hZ8Ftwh8kXD14U7JlR0WRQo', # 274038499@qq.com
        'qrJoLzvIQqU1ygqtzWDw2JC59zkBsPqSzLLIeRSxLHA', # 562040899@qq.com
        'UtEqIoCXWJN5XiyfgpXJ7gg23LgOu7jVQjl36Xg7UmQ', # wgj0905@hku.hk
        'sHyni9kN5Elhtrmb3Z10f5USPyyn9o0snPCzIqagafM' # liuhuan19951021@163.com
        ]

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
Current key: Vo6Oh77LXmw5HXMu691RKQIZG03nY1zB9rsVTkhEHqE 1


In [2]:
df = pd.read_csv('./reindex_df.csv')
print(df.shape)

(237671, 7)


In [3]:
df.head()

,index,song,year,artist,genre,lyrics,word_count
0,0,louder-flux-pavilion-doctor-p-remix,2012,dj-fresh,Electronic,I can't control this feeling Something's happe...,341.0
1,1,that-s-my-name,2009,akcent,Pop,In my heart I will keep you In my heart Foreve...,318.0
2,2,lemonade,2007,apologetix,Rock,"Oh, Mama, I've been cleared of my crimes and I...",250.0
3,3,return-of-the-hustle,2007,fabolous,Hip-Hop,Record mode! (Record mode!) Get your money in ...,681.0
4,4,so-sad,1974,george-harrison,Rock,Now the winter has come To eclipse out the sun...,163.0


In [3]:
emo_path = r'./lyrics_emotion.json'

In [4]:
emo_path_out = r'./lyrics_emotion.json'
emo_path_error = r'lyrics_emotion_fixed_error'


def update_emo(index, emo):
    index = index.tolist()[0]
    emo['time'] = str(datetime.now())
    try:
        with open(emo_path_out, 'r') as fn:
            emo_dict = json.load(fn)
        print(index)
        print(emo)
        emo_dict[index] = emo
    except Exception as e:
        emo_dict = {}
        emo_dict[index] = emo
    with open(emo_path_out, 'w') as fp:
        json.dump(emo_dict, fp)
    print(index, 'done')
    return index

def update_error(index):
    with open(emo_path_error, 'a') as fn:
        fn.write(str(index) + '\n')

In [6]:
with open(emo_path, 'r') as fn:
    emo_dict = json.load(fn)
  
lyric_emo = df['lyrics'].tolist()
index_list = df.index.tolist()

total = len(index_list)
error_count = 0
key_count = 0

# for i in range(0, len(df)):
#     pass
for k, v in emo2.items():
    randtime = randint(0, 4)
    if 'emotion' not in v:
        lyrics = df.loc[df.index == int(k)]['lyrics']
        index = df.loc[df.index == int(k)]['index']
#         print(k)
#         print(v)
#         break
        try:
            time.sleep(randtime)
            response = paralleldots.emotion(lyrics)
            if 'code' in response and key_count < len(keys):
                key_count += 1
                print('Current key:', keys[key_count], key_count + 1)
                paralleldots.set_api_key(keys[key_count])
                response = paralleldots.emotion(lyrics)
            elif key_count == len(keys):
                print('Stop at', str(index))
                update_error(str(index) + '\tStop.')
                break
            elif 'message' in response:
                printt('Stop at', str(index))
                update_error(str(index) + '\t' + str(response['message']))
                break
            elif 'Error' in response:
                print('Stop at', str(index))
                update_error(str(index) + '\t' + str(response))
                break
            elif 'emotion' not in response:
                print('Stop at', str(index))
                update_error(str(index) + '\t' + str(response))
                break
            finished_index = update_emo(index, response)
    #         print(finished_index, 'done: ', str(round(((index-1)/total)*100, 5)), str(datetime.now()))
        except Exception as e:
            update_error(str(index) + '\t' + str(e))
            error_count += 1
            raise e
    #         print(e)
#             pass
#     elif 

57591
{'emotion': {'Fear': 0.3478189009, 'Sad': 0.064125222, 'Bored': 0.0847127163, 'Happy': 0.2378346914, 'Excited': 0.1842108163, 'Angry': 0.0812976532}, 'time': '2020-04-30 10:03:37.139202'}
57591 done
57815
{'emotion': {'Fear': 0.1306483157, 'Sad': 0.0918889361, 'Bored': 0.0900784772, 'Happy': 0.2413221578, 'Excited': 0.3154463353, 'Angry': 0.1306157779}, 'time': '2020-04-30 10:03:40.326837'}
57815 done
58970
{'emotion': {'Fear': 0.1429350827, 'Sad': 0.0291387182, 'Bored': 0.1665469601, 'Happy': 0.1512509848, 'Excited': 0.2438094768, 'Angry': 0.2663187773}, 'time': '2020-04-30 10:03:45.433207'}
58970 done


In [14]:
with open(emo_path_out, 'r') as fn:
    emo_fixed = json.load(fn)

In [17]:
for k, v in emo_fixed.items():
    if 'emotion' not in v:
        print(k)
        print(v)

In [10]:
len(emo_fixed)

72852

In [41]:
emo_fixed['4997']

{'emotion': {'Fear': 0.2107523579,
  'Sad': 0.2591371423,
  'Bored': 0.0815402242,
  'Happy': 0.2017061049,
  'Excited': 0.0812641845,
  'Angry': 0.1655999862},
 'time': '2020-04-27 16:38:49.211029'}

In [18]:
len(emo_fixed)

70235

In [11]:
no_id = []

for i in range(len(emo2)):
    try:
        error_i = emo2[str(i)]
        # print(error_i)
    except Exception as e:
        print(e)
        no_id.append(i)

In [16]:
len(no_id)

0

In [13]:
total = len(index_list)
error_count = 0
key_count = 0

# for i in range(0, len(df)):
#     pass
for k in no_id:
    randtime = randint(0, 4)
    # if 'emotion' not in v:
    lyrics = df.loc[df.index == int(k)]['lyrics']
    index = df.loc[df.index == int(k)]['index']
    print(k)
#         print(v)
#         break
    try:
        time.sleep(randtime)
        response = paralleldots.emotion(lyrics)
        if 'code' in response and key_count < len(keys):
            key_count += 1
            print('Current key:', keys[key_count], key_count + 1)
            paralleldots.set_api_key(keys[key_count])
            response = paralleldots.emotion(lyrics)
        elif key_count == len(keys):
            print('Stop at', str(index))
            update_error(str(index) + '\tStop.')
            break
        elif 'message' in response:
            printt('Stop at', str(index))
            update_error(str(index) + '\t' + str(response['message']))
            break
        elif 'Error' in response:
            print('Stop at', str(index))
            update_error(str(index) + '\t' + str(response))
            break
        elif 'emotion' not in response:
            print('Stop at', str(index))
            update_error(str(index) + '\t' + str(response))
            break
        finished_index = update_emo(index, response)
#         print(finished_index, 'done: ', str(round(((index-1)/total)*100, 5)), str(datetime.now()))
    except Exception as e:
        update_error(str(index) + '\t' + str(e))
        error_count += 1
        raise e
    #         print(e)
#             pass
#     elif 

 0.0250078808, 'Sad': 0.0355582964, 'Bored': 0.0058707759, 'Happy': 0.6934575512, 'Excited': 0.1813129481, 'Angry': 0.0587925476}, 'time': '2020-04-30 10:10:04.484307'}
59055 done
59056
59056
{'emotion': {'Fear': 0.2288479083, 'Sad': 0.2487068113, 'Bored': 0.0503141283, 'Happy': 0.0963975485, 'Excited': 0.1169697499, 'Angry': 0.2587638538}, 'time': '2020-04-30 10:10:06.952604'}
59056 done
59057
59057
{'emotion': {'Fear': 0.1497538577, 'Sad': 0.1934457862, 'Bored': 0.0358521323, 'Happy': 0.2769464065, 'Excited': 0.1750467492, 'Angry': 0.1689550682}, 'time': '2020-04-30 10:10:11.776568'}
59057 done
59058
59058
{'emotion': {'Fear': 0.0365393262, 'Sad': 0.1343393019, 'Bored': 0.0174180191, 'Happy': 0.6600152054, 'Excited': 0.1364156342, 'Angry': 0.0152725132}, 'time': '2020-04-30 10:10:14.164588'}
59058 done
59059
59059
{'emotion': {'Fear': 0.1271051307, 'Sad': 0.2182603411, 'Bored': 0.0236876736, 'Happy': 0.4128167622, 'Excited': 0.0934152783, 'Angry': 0.1247148141}, 'time': '2020-04-30 1

In [5]:
with open('./lyrics_emotion_old.json', 'r') as f:
    emo1 = json.load(f)

In [6]:
len(emo1)

73031

In [12]:
with open('./lyrics_emotion.json', 'r') as f:
    emo2 = json.load(f)

In [13]:
len(emo2)

74413

In [14]:
emo2.update(emo1)

In [15]:
len(emo2)

74592

In [12]:
max(list(map(lambda x: int(x), list(emo2.keys()))))

73030

In [16]:
with open('./lyrics_emotion.json', 'w') as f:
    json.dump(emo2, f)